In [ ]:
from music21 import *

# Function to analyze key changes in the score
# Parameters:
# - score: A music21 stream object representing the musical score.
# Returns:
# - loc: A set of measure numbers where key changes occur.
# Example usage:
#   loc = anaKey(score)
def anaKey(score):
    num_measures = sum(1 for _ in score.parts[0].getElementsByClass('Measure'))
    prevKey = score.measures(1,4).analyze('key')
    kc = "KEY CHANGE"
    loc = set()
    prevKC = 1

    for n in range(2, num_measures-3):
        seg = score.measures(n, n+3)
        key = seg.analyze('key')
        if key != prevKey:
            for m in range(prevKC, n+3):
                cur = score.measure(m)
                mKey = cur.analyze('key')
                if mKey == key:
                    print(f"At measure {m} prevKey: {prevKey} curKey: {key}")
                    first_note_or_rest = cur.flat.notes.stream().first() or cur.flat.rests.stream().first()
                    first_note_or_rest.lyric = kc
                    loc.add(m)
                    prevKey = key
                    prevKC = m
                    break
    
    return loc

# Function to condense a score by combining notes into chords
# Parameters:
# - score: A music21 stream object representing the musical score.
# Returns:
# - xx: A condensed version of the score where all parts are combined into a single chordal structure.
# Example usage:
#   condensed_score = condense(score)
def condense(score):    
    xx = score.chordify()
    for c in xx.recurse().getElementsByClass(chord.Chord):
        c.closedPosition(forceOctave=4, inPlace=True)
    return xx

# Function to move the highest note of a chord down by an octave
# Parameters:
# - c: A music21 chord object.
# Returns:
# - new_chord: The chord with the highest note transposed down by an octave.
# Example usage:
#   new_chord = moveChord(chord_obj)
def moveChord(c: chord.Chord) -> chord.Chord:
    if not c.pitches:
        return c
    sorted_notes = sorted(c.pitches)
    c = chord.Chord(sorted_notes)
    highest_pitch = c.pitches[-1]
    transposed_highest = highest_pitch.transpose(interval.Interval("-P8"))
    new_pitches = list(c.pitches[:-1]) + [transposed_highest]
    new_chord = chord.Chord(new_pitches)
    return new_chord

# Function to modify a chord if it contains a major or minor 2nd or 7th interval
# Parameters:
# - ch: A music21 chord object.
# Returns:
# - ch: The chord after modification.
# Example usage:
#   modified_chord = min7(chord_obj)
def min7(ch: chord.Chord) -> chord.Chord:
    intervals = {}
    pitches = sorted(ch.pitches)
    r = 0
    y = 0
    for i in range(len(pitches)):
        for j in range(i + 1, len(pitches)):
            inter = interval.Interval(pitches[i], pitches[j])
            if inter.name == "M2" or inter.name == "m2":
                intervals[f"{pitches[i].nameWithOctave} to {pitches[j].nameWithOctave}"] = inter.name
                r = j
                y = 2
            elif inter.name == "M7" or inter.name == "m7":
                intervals[f"{pitches[i}.nameWithOctave} to {pitches[j].nameWithOctave}"] = inter.name
                r = i
                y = 7
    if len(intervals) == 1:
        if y == 2:
            for k in range(len(pitches)-r):
                ch = moveChord(ch)
        if y == 7:
            for k in range(len(pitches)-r):
                ch = moveChord(ch)
        ch = ch.transpose("P8")
    
    return ch

# Function to adjust a chord's range so that the lowest pitch falls between C4 and C5
# Parameters:
# - ch: A music21 chord object.
# Returns:
# - ch: The chord transposed to the target range if necessary.
# Example usage:
#   adjusted_chord = adjust_chord_range(chord_obj)
def adjust_chord_range(ch):
    if not ch.pitches:
        return ch
    lower_bound = pitch.Pitch("C4")
    upper_bound = pitch.Pitch("C5")

    lowest_pitch = min(ch.pitches)

    if lowest_pitch < lower_bound:
        ch = ch.transpose("P8")
    elif lowest_pitch > upper_bound:
        ch = ch.transpose("-P8")

    return ch

# Function to convert a set of pitches to a Roman numeral based on the current key
# Parameters:
# - set: A set of pitches (music21.pitch).
# - curKey: The current key (music21.key.Key).
# Returns:
# - RomanNumeral: A Roman numeral object representing the chord.
# Example usage:
#   roman_numeral = notesToRoman(pitches_set, curKey)
def notesToRoman(set:pitch, curKey):
    ch = chord.Chord(list(set))
    if roman.romanNumeralFromChord(ch, curKey).functionalityScore == 0 or roman.romanNumeralFromChord(ch, curKey.parallel).functionalityScore == 0:
        x = 1
        chh = moveChord(ch)
        while x < len(ch.pitches):
            if roman.romanNumeralFromChord(chh, curKey).functionalityScore > 0 or roman.romanNumeralFromChord(ch, curKey.parallel).functionalityScore > 0:
                ch = chh
                x = 100000000
            else:
                chh = moveChord(chh)
                x += 1
        if x < 1000:
            ch = min7(ch)
                
    return roman.romanNumeralFromChord(adjust_chord_range(ch), curKey)  

# Function to analyze harmony and key changes, inserting Roman numeral annotations
# Parameters:
# - score: The original music21 stream object representing the score.
# - condensed: A condensed version of the score (output of condense function).
# - loc: A set of measure numbers where key changes occur (output of anaKey function).
# Returns:
# - outPart: A music21 stream object with Roman numeral annotations and key changes.
# Example usage:
#   analyzed_part = anaHarm(score, condensed_score, key_change_locations)
def anaHarm(score, condensed, loc): 
    outPart = condensed.template(fillWithRests=False)
    curKey = score.analyze('key')

    inMeasures = list(condensed[stream.Measure])
    outMeasures = list(outPart[stream.Measure])

    for i in range(sum(1 for _ in score.parts[0].getElementsByClass('Measure'))):
        if i+1 in loc:
            curKey = score.measure(i+1).analyze('key')
        inMeasure = inMeasures[i]
        outMeasure = outMeasures[i]
        dur = inMeasure.duration.quarterLength

        for beatStart in range(int(dur)):
            combined_pitches = set()
            beatNotes = inMeasure.getElementsByOffset(beatStart,
                                                    beatStart + 1,
                                                    includeEndBoundary=False
                                                    ).getElementsByClass(chord.Chord)
            for elem in beatNotes:
                if isinstance(elem, chord.Chord):
                    if (elem.offset) == int(elem.offset):
                        combined_pitches.update(elem.pitches)
            beatRN = notesToRoman(combined_pitches, curKey)
            beatRN.lyric = beatRN.figure
            outMeasure.insert(beatStart, beatRN)
        
    for m in loc:
        mod = expressions.TextExpression("MODULATION")
        k = score.measure(m).analyze('key')
        newKey = expressions.TextExpression("new key: " + str(k))
        outPart.measure(m-1).append(newKey)
    
    return outPart

# Function to run the full analysis on a score, combining key change and harmonic analysis
# Parameters:
# - score: A music21 stream object representing the musical score.
# Returns:
# - outPart: A music21 stream object with key change and harmonic analysis annotations.
# Example usage:
#   analyzed_score = analyze(score)
def analyze(score):
    loc = anaKey(score)
    xx = condense(score)
    outPart = anaHarm(score, xx, loc)
    return outPart
